In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix
import gc

datasets_path = '/content/drive/My Drive/m5-forecasting-accuracy/SilverCode(final)/datasets/'
pred_path = '/content/drive/My Drive/m5-forecasting-accuracy/SilverCode(final)/Predict/'
sub_path = '/content/drive/My Drive/m5-forecasting-accuracy/SilverCode(final)/Predict/sub/'

In [ ]:
# 读取按商店预测的预测文件
per_store_pred = pd.read_csv(pred_path + 'per_store_pred(lgb).csv')
# 分别读取每个商店的预测
CA1_final = per_store_pred[per_store_pred['id'].str.endswith('CA_1_evaluation')].reset_index(drop=True)
CA2_final = per_store_pred[per_store_pred['id'].str.endswith('CA_2_evaluation')].reset_index(drop=True)
CA3_final = per_store_pred[per_store_pred['id'].str.endswith('CA_3_evaluation')].reset_index(drop=True)
CA4_final = per_store_pred[per_store_pred['id'].str.endswith('CA_4_evaluation')].reset_index(drop=True)

TX1_final = per_store_pred[per_store_pred['id'].str.endswith('TX_1_evaluation')].reset_index(drop=True)
TX2_final = per_store_pred[per_store_pred['id'].str.endswith('TX_2_evaluation')].reset_index(drop=True)
TX3_final = per_store_pred[per_store_pred['id'].str.endswith('TX_3_evaluation')].reset_index(drop=True)

WI1_final = per_store_pred[per_store_pred['id'].str.endswith('WI_1_evaluation')].reset_index(drop=True)
WI2_final = per_store_pred[per_store_pred['id'].str.endswith('WI_2_evaluation')].reset_index(drop=True)
WI3_final = per_store_pred[per_store_pred['id'].str.endswith('WI_3_evaluation')].reset_index(drop=True)

# 读取按州预测的预测文件
per_state_pred = pd.read_csv(pred_path + 'per_state(WI+CA)_pred.csv').drop(columns = ['Unnamed: 0'])

CA1_state_final = per_state_pred[per_state_pred.id.str.endswith('CA_1_evaluation')].reset_index(drop=True)
CA2_state_final = per_state_pred[per_state_pred.id.str.endswith('CA_2_evaluation')].reset_index(drop=True)
CA3_state_final = per_state_pred[per_state_pred.id.str.endswith('CA_3_evaluation')].reset_index(drop=True)
CA4_state_final = per_state_pred[per_state_pred.id.str.endswith('CA_4_evaluation')].reset_index(drop=True)

WI1_state_final = per_state_pred[per_state_pred.id.str.endswith('WI_1_evaluation')].reset_index(drop=True)
WI2_state_final = per_state_pred[per_state_pred.id.str.endswith('WI_2_evaluation')].reset_index(drop=True)
WI3_state_final = per_state_pred[per_state_pred.id.str.endswith('WI_3_evaluation')].reset_index(drop=True)

In [ ]:
## 按照a榜的分数加权地去store和state的平均，CA2融合分数不好
CA1_merge_final = CA1_final.copy()
CA3_merge_final = CA3_final.copy()
CA4_merge_final = CA4_final.copy()

WI1_merge_final = WI1_final.copy()
WI2_merge_final = WI2_final.copy()
WI3_merge_final = WI3_final.copy()

CA1_merge_final[[f'F{col}' for col in range(1,29)]] = (0.47035/(0.47123+0.47035))*CA1_final[[f'F{col}' for col in range(1,29)]] + \
        (0.47123/(0.47123+0.47035))*CA1_state_final[[f'F{col}' for col in range(1,29)]]

CA3_merge_final[[f'F{col}' for col in range(1,29)]] = (0.47100/(0.47059+0.47100))*CA3_final[[f'F{col}' for col in range(1,29)]] + \
        (0.47059/(0.47059+0.47100))*CA3_state_final[[f'F{col}' for col in range(1,29)]]

CA4_merge_final[[f'F{col}' for col in range(1,29)]] = (0.47078/(0.47064+0.47078))*CA4_final[[f'F{col}' for col in range(1,29)]] + \
        (0.47064/(0.47064+0.47078))*CA4_state_final[[f'F{col}' for col in range(1,29)]]

WI1_merge_final[[f'F{col}' for col in range(1,29)]] = (0.47114/(0.47114+0.47105))*WI1_final[[f'F{col}' for col in range(1,29)]] + \
        (0.47105/(0.47114+0.47105))*WI1_state_final[[f'F{col}' for col in range(1,29)]]

WI2_merge_final[[f'F{col}' for col in range(1,29)]] = (0.4696/(0.4696+0.47010))*WI2_final[[f'F{col}' for col in range(1,29)]] + \
        (0.47010/(0.4696+0.47010))*WI2_state_final[[f'F{col}' for col in range(1,29)]]

WI3_merge_final[[f'F{col}' for col in range(1,29)]] = (0.47084/(0.47222+0.47084))*WI3_final[[f'F{col}' for col in range(1,29)]] + \
        (0.47222/(0.47084+0.47222))*WI3_state_final[[f'F{col}' for col in range(1,29)]]

all_preds1 = pd.concat([CA1_merge_final,CA2_final,CA3_merge_final,CA4_merge_final,TX1_final,TX2_final,
             TX3_final,WI1_merge_final,WI2_merge_final,WI3_merge_final],axis=0).reset_index(drop=True) 

In [ ]:
## 制作提交文件 ##
all_preds = all_preds1
submission = pd.read_csv(datasets_path + 'sample_submission.csv')[['id']]
submission = submission.merge(all_preds, on=['id'], how='left').fillna(0) # 前面_validation填充为0
submission.to_csv(sub_path + 'Sub_merge_stores_states(lgb).csv', index=False)

# 记录：
# per_store_pred(lgb).csv + per_state(WI+CA)_pred.csv = Sub_merge_stores_states(lgb).csv 融合得到的分数为0.61704，比最终提交的B榜分数0.61940提升一些

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,HOBBIES_1_002_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,HOBBIES_1_003_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,HOBBIES_1_004_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,HOBBIES_1_005_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0.499919,0.471186,0.437475,0.447991,0.536921,0.557289,0.551727,0.530468,0.498270,0.485808,0.641237,0.668622,0.701426,0.816980,0.650020,0.552532,0.585845,0.615315,0.568401,0.733350,0.856588,0.596416,0.620825,0.596342,0.471061,0.524273,0.578633,0.682655
60976,FOODS_3_824_WI_3_evaluation,0.264173,0.257155,0.243765,0.224255,0.230381,0.283615,0.262848,0.274286,0.263021,0.280638,0.377941,0.342652,0.348095,0.407123,0.358798,0.326062,0.357648,0.346441,0.265864,0.390027,0.422074,0.318443,0.371583,0.392433,0.282350,0.290324,0.320441,0.344264
60977,FOODS_3_825_WI_3_evaluation,0.707537,0.542472,0.504926,0.476878,0.540831,0.639085,0.645888,0.648516,0.550079,0.724970,1.062464,1.170589,0.974054,1.231085,1.205026,0.827803,1.098170,1.046923,0.783028,1.158347,1.289886,1.007816,1.107737,1.099633,0.751401,0.722390,0.758889,0.965955
60978,FOODS_3_826_WI_3_evaluation,1.014188,1.029115,0.990539,0.964949,1.054325,1.151373,1.058109,1.148868,1.070374,0.972573,1.272943,1.378673,1.311373,1.411144,1.308165,1.163874,1.154940,1.223891,1.068682,1.445418,1.481660,1.171479,1.364837,1.390009,1.115929,1.206271,1.233813,1.260121
